# Selecting SRM Targets with MSstats

[No more coefficients of variance](https://yaaminiv.github.io/Selecting-SRM-Targets-2/)! Let's use MSstats instead! This method is more statistically sound, and Emma has used a similar method to analyze DDA data for potential targets. I'm basing my workflow off of the [MSstats manual](https://bioconductor.org/packages/release/bioc/vignettes/MSstats/inst/doc/MSstats-manual.pdf).

## Step 1: Merge Normalized Skyline Output with GO Terms

Before I go into MSstats, I first want to see which proteins have associated annotations. If a protein doesn't have an annotation, it's not going to be useful for me!

I used [Galaxy](usegalaxy.org) to merge the [protein annotations]() with my [normalized Skyline output](https://github.com/RobertsLab/project-oyster-oa/blob/master/analyses/DNR_Skyline_20170511/2017-05-11-normalized-peak-areas.csv).

The first thing I had to do was convert the Skyline output, a .csv file, to a tab-delimited one.

![untitled](https://cloud.githubusercontent.com/assets/22335838/26018881/718ee2a4-3726-11e7-843c-a6d0aa07e5ef.png)

I downloaded the [tab-delimited file](https://github.com/RobertsLab/project-oyster-oa/blob/master/analyses/DNR_Skyline_20170512/2017-05-12-normalized-peak-areas.tabular) to delete the column header. I reuploaded it to Galaxy afterwards.

![untitled](https://cloud.githubusercontent.com/assets/22335838/26019684/2fe7f7ea-372c-11e7-93aa-5609c8de344a.png)

Then, I merged the two columns with protein names in the table with GO terms in Excel. And uploaded the file to Galaxy. The resultant file is [here]().

Finally, I joined the two files based on protein names.



## Step 1: Add MSstats tool to Skyline

Under Tools >> Tool Store, I selected MSstats. Because the Windows computer doesn't have R either, Skyline is installing MSstats, R and all associated packages.

![image-2](https://cloud.githubusercontent.com/assets/22335838/26008020/ed36b532-36f7-11e7-944f-61d1ef74ff26.png)
![image-3](https://cloud.githubusercontent.com/assets/22335838/26008021/ed4f9548-36f7-11e7-89b9-66988cce4093.png)

## Step 2: Export Skyline data

According to the manual, I need the following variables from Skyline: ProteinName, PeptideSequence, PrecursorChange, FragmentIon, ProductCharge, IsotopeLabelType, Condition, BioReplicate, Run and Intensity. All variable names should be fixed, and any missing data should be replaced with "NA."

### Step 2a: Add Condition and BioReplicate information

For this step, I'm adapting information from [Emma's lab notebook](https://www.evernote.com/shard/s242/sh/6beda768-71df-477c-8e50-5400c8bfc0d9/641c7a94006329064c9595052e47097f). Condition would refer to the combination of sample site and eelgrass presence. BioReplicate would either be 1 or 2, since I ran each sample in duplicate.

**Table 1**. Condition and BioReplicate information for MSstats.

| ID | DNR Vial | Sample Site | Eelgrass Presence | Replicate |
|:--:|:--------:|:-----------:|:-----------------:|:---------:|
|  1 |   O127   |      WB     |        Bare       |     1     |
|  2 |   O107   |      SK     |      Eelgrass     |     1     |
|  3 |    O07   |      CI     |      Eelgrass     |     1     |
|  4 |    O77   |      PG     |      Eelgrass     |     1     |
|  5 |    O47   |      FB     |        Bare       |     1     |
|  6 |    O55   |      PG     |        Bare       |     1     |
|  7 |    O37   |      FB     |      Eelgrass     |     1     |
|  8 |    O15   |      CI     |        Bare       |     1     |
|  9 |   O142   |      WB     |      Eelgrass     |     1     |
| 10 |   O119   |      SK     |        Bare       |     1     |
| 11 |    O47   |      FB     |        Bare       |     2     |
| 12 |   O127   |      WB     |        Bare       |     2     |
| 13 |    O37   |      FB     |      Eelgrass     |     2     |
| 14 |    O55   |      PG     |        Bare       |     2     |
| 15 |    O15   |      CI     |        Bare       |     2     |
| 16 |   O119   |      SK     |        Bare       |     2     |
| 18 |   O142   |      WB     |      Eelgrass     |     2     |
| 19 |    O07   |      CI     |      Eelgrass     |     2     |
| 20 |    O77   |      PG     |      Eelgrass     |     2     |
| 24 |   O107   |      SK     |      Eelgrass     |     2     |
| 25 |   O107   |      SK     |      Eelgrass     |     3     |

Under Settings >> Annotation, I checked the box for "Condition" and then clicked "Edit List." I created a "Values List," and added the following conditions and clicked "OK":

- O15-CI-Bare
- O07-CI-Eelgrass
- O47-FB-Bare
- O37-FB-Eelgrass
- O55-PG-Bare
- O77-PG-Eelgrass
- O119-SK-Bare
- O107-SK-Eelgrass
- O127-WB-Bare
- O142-WB-Eelgrass

I then checked the box for "BioReplicate" and clicked "OK":

Under View >> Results Grid, I added the Condition and BioReplicate information to each sample based on Table 1.

## Step 2b: Export Report from Skyline